# Parallelizing operations using Multithreading, Multiprocessing and Distributed processing

Multithreading, Multiprocessing and Distributed processing are three common methods to parallelize your workloads to reduce the execution. Imagine a scenario where you've to train thousands of ML models or inference a model for thousands of records. Writing this operation as loop runs this sequentially and might take a long time to finish. The operation can be parallelized to reduce the execution time.


Each method is suitable for certain types of problems.

**Multithreading**: For I/O operations, API calls - reading hundreds of files, making large number of API calls.

**Multiprocessing**: For CPU intensive operations - Large math/statistical operations including training large number of ML models.

**Distributed processing**: Same as multiprocessing, but when your operations are too long.


**Note: Some of the concepts in the notebook is oversimplified for ease of explanation. Please refer the additonal materials to get deeper understanding of the concepts.**

[Multithreading vs. Multiprocessing in Python
](https://towardsdatascience.com/multithreading-vs-multiprocessing-in-python-3afeb73e105f)

[What Is Distributed Computing?
](https://hazelcast.com/glossary/distributed-computing/#:~:text=Distributed%20computing%20(or%20distributed%20processing,and%20to%20coordinate%20processing%20power.)

[Apache Spark: A Conceptual Orientation](https://towardsdatascience.com/apache-spark-a-conceptual-orientation-e326f8c57a64)

## Implementation

The notebook uses a python module `joblib` for parallelizing operations. `joblib` supports all three types of parallelization with a minor code change. The syntax for using joblib is relatively simple and it comes with optmizations for numpy arrays.

In [ ]:
!pip install joblib==1.1.0 joblibspark==0.5.0

Lets take a simple sequential code involving large number of operations and see how you can convert the code to use joblib.  The following is a list comprehesion in Python to find the square root of first 10K numbers.

In [1]:
from math import sqrt
results = [sqrt(i) for i in range(10000)]

print(results[-2:])

[99.98999949994999, 99.99499987499375]


Lets convert this code to use joblib

In [2]:
from math import sqrt
from joblib import Parallel, delayed

results = Parallel()(delayed(sqrt)(i) for i in range(10000))

print(results[-2:])

[99.98999949994999, 99.99499987499375]


All we did was wrapping the sqrt function with `Parallel` and `delayed` functions of `joblib`.

## Multiprocessing

Multiprocessing is the technique using which you execute your operations using a seperate dedicated Python process. By default your application runs within a single process. Each Python process uses a CPU in the machine. By using multiple processes you're able to use all the CPUs in the machine.

Let's generate some dummy data for training some models.

In [7]:
from sklearn import datasets
import numpy as np
import pandas as pd

data_size = 1000000

X, y = datasets.make_classification(n_samples=data_size, n_features=10, n_redundant=0)
df = pd.DataFrame(data=X)
df["target"] = y
groups = np.random.choice(1000, size=data_size)
df["group"] = groups

The `train_model` function accepts a pandas Dataframe input and trains a model on the data.

In [8]:
from sklearn.ensemble import RandomForestClassifier

def train_model(data: pd.DataFrame) -> RandomForestClassifier:
    """
    Trains an ML model for with the input data
    """
    X = data.drop(["target", "group"], axis=1)
    y = data["target"]
    clf = RandomForestClassifier(random_state=0)
    clf.fit(X, y)
    return clf

Let's train a model for each group of data we've in the source data. Here the models are trained sequentially without any parallelism.

In [5]:
%%time

models = [train_model(group) for _, group in df.groupby("group")]

CPU times: user 4min 20s, sys: 1.68 s, total: 4min 21s
Wall time: 4min 21s


The entire training process took 1 min 40s. Now let's try it using multiprocessing. Notice the `prefer=processes`, this is the parameter that instructs joblib to use `multiprocessing`. `n_jobs=-2` means it should use all the cpus in the system except one (so that rest of system processes can run).

In [6]:
%%time

from joblib import Parallel, delayed
from multiprocessing import cpu_count

models = Parallel(n_jobs=-2, prefer="processes")(delayed(train_model)(group) for _, group in df.groupby("group"))

CPU times: user 10.9 s, sys: 2.01 s, total: 12.9 s
Wall time: 58.9 s


The operation finished under a minute!

## Distributed processing

Multiprocessing uses all the CPUs in a single machine to parallelize workloads. Distributed processing uses multiple machines to parallelize the workloads to get even better performance. Distributed processing in joblib is implemented with the help of Spark.

**Note: Distributed processing comes with a considerable overhead. Unless your operations are too slow (execution time over an hour) it shouldn't be used. Using it for short workloads would increase the execution time**

Create a Spark session session. Here the `spark.kubernetes.container.image` is a Spark executor image with scikit-learn built in. Use the correct image based on the Spark version you're using:

* `3.1.2`: 555157090578.dkr.ecr.us-east-1.amazonaws.com/datascience/mlops/jupyterhub/spark-executor-jupyter:scikit-learn-3-1-2
* `3.2.0`: 555157090578.dkr.ecr.us-east-1.amazonaws.com/datascience/mlops/jupyterhub/spark-executor-jupyter:scikit-learn-3-2-0

**Warning**: These images are built for just this example code. 

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, col

conf = SparkConf()  # create the configuration
conf.set(
    "spark.kubernetes.container.image",
    "555157090578.dkr.ecr.us-east-1.amazonaws.com/datascience/mlops/jupyterhub/spark-executor-jupyter:scikit-learn-3-1-2"
)
conf.set("spark.rpc.message.maxSize", 1024)

spark = SparkSession.builder.config(conf=conf).getOrCreate()

22/05/12 06:32:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/12 06:32:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/05/12 06:32:23 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


Register the Spark session with joblib

In [4]:
from joblibspark import register_spark
register_spark()

Change the backend to `spark` and set `n_jobs=50` (this is an ideal number based on our Spark platform).

In [9]:
%%time

from joblib import Parallel, delayed

models = Parallel(n_jobs=50, backend="spark")(delayed(train_model)(group) for _, group in df.groupby("group"))

/home/axj1011/env/fitter/lib/python3.9/site-packages/joblibspark/backend.py:107: UserWarning: User-specified n_jobs (50) is greater than the max number of concurrent tasks (3) this cluster can run now.If dynamic allocation is enabled for the cluster, you might see more executors allocated.
  warnings.warn(f"User-specified n_jobs ({n_jobs}) is greater than the max number of "


CPU times: user 7.53 s, sys: 2.13 s, total: 9.66 s
Wall time: 1min 3s


# Multithreading

Let's try a scenario for multithreading. We've to make 1000 REST API calls to retrieve data from an external system.

In [10]:
import requests

def make_request(url):
    r = requests.get(url, allow_redirects=True)
    r.raise_for_status()
    return r.text

In [11]:
urls = ["https://www.google.com"] * 500

Let's make the API calls sequentially first

In [12]:
%%time

responses = [make_request(url) for url in urls]

CPU times: user 4.98 s, sys: 286 ms, total: 5.26 s
Wall time: 1min 28s


Let's make the same API calls parallely using multithreading. Notice the `prefer="threads"` which instructs joblib to use multithreading.

In [14]:
%%time

from joblib import Parallel, delayed

responses = Parallel(n_jobs=20, prefer="threads")(delayed(make_request)(url) for url in urls)

CPU times: user 6.12 s, sys: 407 ms, total: 6.53 s
Wall time: 5.2 s
